In [10]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Data
english_words = ['hello', 'world', 'thank', 'you', 'good', 'morning', 'night', 'food', 'water', 'love']
french_words = ['bonjour', 'monde', 'merci', 'vous', 'bon', 'matin', 'nuit', 'nourriture', 'eau', 'amour']

# Tokenize the words
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(english_words)
eng_sequences = tokenizer_eng.texts_to_sequences(english_words)

tokenizer_fr = Tokenizer()
tokenizer_fr.fit_on_texts(french_words)
fr_sequences = tokenizer_fr.texts_to_sequences(french_words)

# Pad sequences
max_seq_length = max(max(len(seq) for seq in eng_sequences), max(len(seq) for seq in fr_sequences))
eng_sequences = pad_sequences(eng_sequences, maxlen=max_seq_length, padding='post')
fr_sequences = pad_sequences(fr_sequences, maxlen=max_seq_length, padding='post')

# Define vocabulary sizes
num_eng_tokens = len(tokenizer_eng.word_index) + 1
num_fr_tokens = len(tokenizer_fr.word_index) + 1

In [11]:
# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding(input_dim=num_eng_tokens, output_dim=128)(encoder_inputs)
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding(input_dim=num_fr_tokens, output_dim=128)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_fr_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()

# Prepare input and output sequences
decoder_input_data = np.zeros_like(fr_sequences)
decoder_input_data[:, :-1] = fr_sequences[:, 1:]
decoder_output_data = np.expand_dims(fr_sequences, -1)

# Train the model
model.fit([eng_sequences, decoder_input_data], decoder_output_data, epochs=500)

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)    │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ input_layer_10 (InputLayer)   │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_5 (Embedding)       │ (None, None, 128)         │           1,408 │ input_layer_9[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ embedding_6 (Embedding)       │ (None, None, 128)         │           1,408 │ input_layer_10[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_4 (LSTM)                 │ [(None, 128), (None,      │         131,584 │ embedding_5[0][0]          │
│                               │ 128), (None, 128)]        │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm_5 (LSTM)                 │ [(None, None, 128),       │         131,584 │ embedding_6[0][0],         │
│                               │ (None, 128), (None, 128)] │                 │ lstm_4[0][1], lstm_4[0][2] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, None, 11)          │           1,419 │ lstm_5[0][0]               │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 267,403 (1.02 MB)

 Trainable params: 267,403 (1.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 2.3979
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3916
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3852
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 2.3788
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.3723
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3657
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 2.3589
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3519
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.3447
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 2.3371
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.3293
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3210
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3123
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.3031
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.2934
Epoch 16/500
1/1 ━━

In [12]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding_input = tf.keras.layers.Embedding(input_dim=num_fr_tokens, output_dim=128)
decoder_embedding_output = decoder_embedding_input(decoder_inputs)
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_embedding_output, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_lstm_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [13]:
def translate(input_text):
    # Tokenize the input text
    input_sequence = tokenizer_eng.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_seq_length, padding='post')

    # Encode the input sequence
    states_value = encoder_model.predict(input_sequence)

    # Generate empty target sequence of length 1 with only the start character
    target_sequence = np.zeros((1, 1))

    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer_fr.index_word[sampled_token_index]
        decoded_sentence += sampled_char + ' '

        # Exit condition: either hit max length or find stop character
        if (sampled_char == ' ' or
            len(decoded_sentence.split()) > max_seq_length or
            sampled_char not in tokenizer_fr.index_word):
            stop_condition = True

        # Update the target sequence
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence.strip()

In [15]:
# Testing the Translation
print("Translation: ")
for word in english_words:
    print(f"English: {word} -> French: {translate(word)}")

Translation: 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
English: hello -> French: bonjour
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
English: world -> French: monde
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
English: thank -> French: merci
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
English: you -> French: vous
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
English: good -> French: bon
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
English: morning -> French: matin
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
English: night -> French: nuit
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
English: food -> French: nourriture
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
English: water -> French: eau
1/1 ━━━━━━━━━━━